In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import time


def connect_robot(ip="192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip)  # IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [2]:
from capture_image import capture_image
import sys
import os
import warnings
import numpy as np

# add parent folder to sys.path
sys.path.insert(0, os.path.abspath(".."))

# ignore DeprecationWarning messages
warnings.filterwarnings("ignore", category=DeprecationWarning)

# ignore RuntimeWarning messages
warnings.filterwarnings("ignore", category=RuntimeWarning)

from volume_estimation.src.models_1_no_vol.predict_full_pipeline import predict_no_vol
from segmentation_and_depth.src.models.predict_model import predict as predict_seg_depth
from volume_estimation.src.models_2_input_vol.predict_full_pipeline import (
    predict_with_vol,
)

In [3]:
rtde_c, rtde_r, rtde_io_set = connect_robot()
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(0)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")

Activating Gripper
Gripper activated


In [4]:
j_start = [-3.141514603291647,
 -0.39998288572344975,
 -1.7599776983261108,
 2.149993820781372,
 -1.5708015600787562,
 -1.5499151388751429]

j_up = [-1.2547920385943812,
 -1.1487035018256684,
 -0.9265894889831543,
 3.4698049265095214,
 -2.3617973963366907,
 -2.868312184010641]

j_bottles_1 = [-0.3581450621234339,
 -1.9314056835570277,
 -1.2249127626419067,
 1.648374004954956,
 -1.5516436735736292,
 -1.853001896535055]

j_bottles_2 = [-0.6685603300677698,
 -2.246812482873434,
 -0.9622687697410583,
 1.3413416582294921,
 -1.867582146321432,
 -2.1364834944354456]

j_miscroscope = [-3.523916546498434,
 -1.4186616402915497,
 -2.1420812606811523,
 3.26844231664624,
 -1.840297047291891,
 -3.248759094868795]

j_flasks_1 = [-3.149257485066549,
 -1.2185992461493989,
 -2.3591017723083496,
 2.720782919520996,
 -1.07640248933901,
 -2.7709713617907923]

j_bottle_holder = [-3.169065777455465,
 -1.727936407128805,
 -2.395404815673828,
 2.8663622575947265,
 -0.6110208670245569,
 -1.927342716847555]

j_flask_holders_trash = [-3.504859511052267,
 -1.2505887907794495,
 -2.19888973236084,
 2.0256845194050292,
 -0.8002708593951624,
 -1.601645294819967]

j_flask_storage_1 = [-4.287753526364462,
 -1.8319250545897425,
 -1.2737982273101807,
 3.1778737741657714,
 0.3764202296733856,
 -3.236988369618551]

j_flask_storage_2 = [-4.287816349660055,
 -1.8942696056761683,
 -1.6332104206085205,
 3.597304506892822,
 0.3639531135559082,
 -3.235102717076437]

j_flask_storage_3 = [-4.288263742123739,
 -2.134270807305807,
 -1.8128883838653564,
 4.010412855739258,
 0.36713337898254395,
 -3.228081766759054]


In [24]:
def check_microscope(path):
    flask_present = False
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)

    # if counts only contains 0, then vessel is closed:
    if len(counts) == 1:
        return flask_present
    if counts[1] > 10000:
        flask_present = True
        return flask_present
    else:
        return flask_present
    
def check_flasks_1(path):
    flask_present = False
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)

    # if counts only contains 0, then vessel is closed:
    if len(counts) == 1:
        return flask_present
    if counts[1] > 5000:
        flask_present = True
        return flask_present
    else:
        return flask_present
    
def check_bottle_holder(path):
    flask_present = False
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)

    # if counts only contains 0, then vessel is closed:
    if len(counts) == 1:
        return flask_present
    if counts[1] > 5000:
        flask_present = True
        return flask_present
    else:
        return flask_present
    
def check_flask_holder_and_trash(path):
    flask_present = False
    trash_present = True
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)
    if len(counts) == 1:
        trash_present = False
        return flask_present, trash_present
    if counts[1] < 5000:
        trash_present = False
        return flask_present, trash_present
    if counts[1] < 28000:
        return flask_present, trash_present
    else:

        flask_present = True
        return flask_present, trash_present
    
        
def check_flask_storage(path):
    flask_1_present = False
    flask_2_present = False
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # split segmentation map in two horizontal parts
    segmentation_map_npy_1 = segmentation_map_npy[:, :240, :]
    segmentation_map_npy_2 = segmentation_map_npy[:, 240:, :]
    # value count of segmentation map
    unique_1, counts_1 = np.unique(segmentation_map_npy_1, return_counts=True)
    unique_2, counts_2 = np.unique(segmentation_map_npy_2, return_counts=True)

    if len(counts_1) > 1:
        if counts_1[1] > 5000:
            flask_1_present = True
    if len(counts_2) > 1:
        if counts_2[1] > 5000:
            flask_2_present = True

    return flask_1_present, flask_2_present 

def check_bottle_input(path):
    media_present = False
    washing_present = False
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # split segmentation map in two horizontal parts
    segmentation_map_npy_1 = segmentation_map_npy[:, :, :280]
    segmentation_map_npy_2 = segmentation_map_npy[:, :, 280:]
    # value count of segmentation map
    unique_1, counts_1 = np.unique(segmentation_map_npy_1, return_counts=True)
    unique_2, counts_2 = np.unique(segmentation_map_npy_2, return_counts=True)

    if len(counts_1) > 1:
        if counts_1[1] > 5000:
            media_present = True
    if len(counts_2) > 1:
        if counts_2[1] > 5000:
            washing_present = True

    return media_present, washing_present


In [25]:
init_q = rtde_r.getActualQ()
init_q

[-3.1414883772479456,
 -0.40001554906878667,
 -1.7599780559539795,
 2.150005980128906,
 -1.5707433859454554,
 -1.5499456564532679]

In [20]:
def check_setup_record(rtde_c):
    rtde_c.moveJ(j_start, 1.5, 1.5)
    rtde_c.moveJ(j_miscroscope, 1.5, 1.5)
    path_microscope = capture_image()

    rtde_c.moveJ(j_flasks_1, 1.5, 1.5)
    path_flasks_1 = capture_image()

    rtde_c.moveJ(j_bottle_holder, 1.5, 1.5)
    path_bottle_holder = capture_image()

    rtde_c.moveJ(j_flask_holders_trash, 1.5, 1.5)
    path_flask_holders_trash = capture_image()

    rtde_c.moveJ(j_flask_storage_1, 1.5, 1.5)
    path_flask_storage_1 = capture_image()

    rtde_c.moveJ(j_flask_storage_2, 1.5, 1.5)
    path_flask_storage_2 = capture_image()

    rtde_c.moveJ(j_flask_storage_3, 1.5, 1.5)
    path_flask_storage_3 = capture_image()

    rtde_c.moveJ(j_start, 1.5, 1.5)

    rtde_c.moveJ(j_up, 1.5, 1.5)

    rtde_c.moveJ(j_bottles_1, 0.7, 0.7)
    rtde_c.moveJ(j_bottles_2, 1.5, 1.5)
    path_bottles_input = capture_image()
    rtde_c.moveJ(j_bottles_1, 1.5, 1.5)
    rtde_c.moveJ(j_up, 0.7, 0.7)

    rtde_c.moveJ(j_start, 1.5, 1.5)

    return path_microscope, path_flasks_1, path_bottle_holder, path_flask_holders_trash, path_flask_storage_1, path_flask_storage_2, path_flask_storage_3, path_bottles_input

def check_setup_analyze(path_microscope, path_flasks_1, path_bottle_holder, path_flask_holders_trash, path_flask_storage_1, path_flask_storage_2, path_flask_storage_3, path_bottles_input):

    flask_microscope_present = check_microscope(path_microscope)

    flask_1_present = check_flasks_1(path_flasks_1)

    bottle_present = check_bottle_holder(path_bottle_holder)

    flask_present, trash_present = check_flask_holder_and_trash(path_flask_holders_trash)

    storage_flask_1_present, storage_flask_2_present = check_flask_storage(path_flask_storage_1)

    storage_flask_3_present, storage_flask_4_present = check_flask_storage(path_flask_storage_2)

    storage_flask_5_present, empty = check_flask_storage(path_flask_storage_3)

    media_present, washing_present = check_bottle_input(path_bottles_input)

    return flask_microscope_present, flask_1_present, bottle_present, flask_present, trash_present, storage_flask_1_present, storage_flask_2_present, storage_flask_3_present, storage_flask_4_present, storage_flask_5_present, media_present, washing_present

def print_results(flask_microscope_present, flask_1_present, bottle_present, flask_present, trash_present, storage_flask_1_present, storage_flask_2_present, storage_flask_3_present, storage_flask_4_present, storage_flask_5_present, media_present, washing_present):
    setup_ready = False

    if flask_microscope_present:
        print("Flask present in microscope holder. Please remove flask.")
    
    if flask_1_present:
        print("Flask present in flask holder. Please remove flask.")
    
    if bottle_present:
        print("Bottle present in bottle holder. Please remove bottle.")

    if flask_present:
        print("Flask present in flask holder. Please remove flask.")
    
    if not trash_present:
        print("No trash present in flask holder. Please add trash bottle.")

    if not media_present:
        print("No media present in bottle storage. Please add media bottle.")
    
    if not washing_present:
        print("No washing solution present in bottle storage. Please add washing solution bottle.")

    # if less than 3 flasks are present in storage, then print warning
    if sum([storage_flask_1_present, storage_flask_2_present, storage_flask_3_present, storage_flask_4_present, storage_flask_5_present]) < 3:
        print("Warning: Less than 3 flasks present in storage.")

    if not flask_microscope_present and not flask_1_present and not bottle_present and not flask_present and trash_present and media_present and washing_present and sum([storage_flask_1_present, storage_flask_2_present, storage_flask_3_present, storage_flask_4_present, storage_flask_5_present]) > 3:
        print("Everything okay.")
        setup_ready = True
    print("")
    print("Present empty flasks: ")
    if storage_flask_1_present:
        print("Flask 1")
    if storage_flask_2_present:
        print("Flask 2")
    if storage_flask_3_present:
        print("Flask 3")
    if storage_flask_4_present:
        print("Flask 4")
    if storage_flask_5_present:
        print("Flask 5")

    return setup_ready

In [26]:
path_microscope, path_flasks_1, path_bottle_holder, path_flask_holders_trash, path_flask_storage_1, path_flask_storage_2, path_flask_storage_3, path_bottles_input = check_setup_record(rtde_c)
flask_microscope_present, flask_1_present, bottle_present, flask_present, trash_present, storage_flask_1_present, storage_flask_2_present, storage_flask_3_present, storage_flask_4_present, storage_flask_5_present, media_present, washing_present = check_setup_analyze(path_microscope, path_flasks_1, path_bottle_holder, path_flask_holders_trash, path_flask_storage_1, path_flask_storage_2, path_flask_storage_3, path_bottles_input)
print_results(flask_microscope_present, flask_1_present, bottle_present, flask_present, trash_present, storage_flask_1_present, storage_flask_2_present, storage_flask_3_present, storage_flask_4_present, storage_flask_5_present, media_present, washing_present)

Flask present in microscope holder. Please remove flask.
Bottle present in bottle holder. Please remove bottle.
No media present in bottle storage. Please add media bottle.

Present empty flasks: 
Flask 1
Flask 2
Flask 4


False